In [21]:
import pandas as pd
from collections import Counter
import re
import os
import glob
import numpy as np
import fitz
import os
import operator
import codecs
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
columns = ['file_name', 'All_Consultant_Names'] #required columns
metadata = pd.DataFrame(columns = columns) #create dataframe

In [23]:
def clean(mystr):
#     my_new_str = re.sub("(\\W| +)"," ", mystr) #remove anything that is not a letter or number
#     my_new_str = re.sub(r'\s+', ' ', my_new_str) #eliminate duplicate whitespaces
#     my_new_str = re.sub(r"\b\d+\b", "", my_new_str)
    my_new_str = re.sub(r"[^a-zA-Z0-9-&]+",' ', mystr) #remove special characters
#     my_new_str = my_new_str.strip()
    return my_new_str

#### Read text from all .txt files in the corpus

In [24]:
texts = {}
texts_before_val = {}
text_files = []
count_reference = []
eco_list = []
bio_list = []
consultants_all = []
consultant_name_two_pages = []
names_pdf = []
author = []
creation_date = []
#consultant_count = Counter()
#eco_count = Counter()
consultant_name_df = pd.read_excel('G:/Post Construction/metadata csvs/consultants.xlsx', encoding = 'utf-8-sig')
consultant_name_lst = consultant_name_df["consultant_name"].tolist()
ecoregion_lst = ['agricultural', 'cropland', 'native prairie', 'grassland', 'residential', 'commercial', 'industrial',
                 'boreal forest', 'mixed forest', 'national park', 'provincial park', 'green space','wetlands',
                  'Watercourse','Tundra','Delta','mountains','mountainous']

# search_eco = '|'.join(r'\b{}\b'.format(c.replace(' ', r'\s+')) for c in [eco_item.lower() for eco_item in ecoregion_lst])
# re_comp_eco = re.compile(r'({})'.format(search_eco)) #reference: https://stackoverflow.com/questions/49261780/finding-a-list-of-all-companies-in-a-txt-file-with-python?rq=1

# search = '|'.join(r'\b{}\b'.format(c.replace(' ', r'\s+')) for c in [item.lower() for item in consultant_name_lst])
# re_comp = re.compile(r'({})'.format(search))

source_text_dir =  os.listdir('G:/Post Construction/PDF_Final_text')
source_pdf_dir = "G:/Post Construction/PCMR_All/"
for file in source_text_dir:
    with open("G:/Post Construction/PDF_Final_text/" + file,'r', encoding = 'utf-8-sig') as corpus:
        text_files.append(file.replace(".txt",""))
        doc = fitz.open(os.path.join(source_pdf_dir,file.replace('.txt','.pdf')))
        names_pdf.append(file.replace('.txt','.pdf'))
        page_content_str = ''
        for count in range(0,2):
            page = doc.loadPage(count)
            page_content = page.getText().lower()
            page_content_str += str(page_content).replace('\n','').replace('\\n','').replace('\\',' ').replace('xa0','').replace('xc2','').strip() 
            page_content_str = clean(page_content_str)
            pdf_metadata = doc.metadata
            creation_date.append(pdf_metadata["creationDate"]) 
            author.append(pdf_metadata["author"])
        consultant_name = []
        for name in consultant_name_lst:
            if name.lower() in page_content_str:
                consultant_name.append(name)
        consultant_name_two_pages.append(consultant_name)
        text = corpus.read().lower()
        text = clean(text)
        texts_before_val[file.replace(".txt","")] = text
        eco_lst = list(set([word for word in text.split() if word.startswith('eco')])) ## check words that starts from eco
        eco_list.append(eco_lst)
        bio_lst = list(set([word for word in text.split() if word.startswith('bio')])) ## check words that starts from bio 
        bio_list.append(bio_lst)
        references_count = text.count("references") #reference: https://stackoverflow.com/questions/38401099/how-to-count-one-specific-word-in-python/38401167
        count_reference.append(references_count)
        if references_count > 3:
            before_third = text.split("references")
            result_before_third = '_'.join(before_third[:3]), '_'.join(before_third[3:])
            texts[file] = result_before_third[0]
        elif references_count == 2 or 3:
            before_second = text.split("references") #reference: https://stackoverflow.com/questions/47927039/reading-a-file-until-a-specific-character-in-python
            result_before_second = '_'.join(before_second[:2]), '_'.join(before_second[2:])  # reference: https://stackoverflow.com/questions/17060039/split-string-at-nth-occurrence-of-a-given-character
            texts[file] = result_before_second[0]
        else:
            texts[file] = text
#     counts = Counter(re.sub(r'\s+', ' ', c) for c in re_comp.findall(text))
#     consultant_count.update(counts)
#     counts_eco = Counter(re.sub(r'\s+', ' ', c) for c in re_comp_eco.findall(text))
#     eco_count.update(counts_eco)
# for keyword, count in sorted(consultant_count.items()):    
#     print('{:80}  {}'.format(keyword, count))
# # for keyword_eco, count_eco in sorted(eco_count.items()):    
# #     print('{:80}  {}'.format(keyword_eco, count_eco))

mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: No default Layer config
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object
mupdf: invalid page object


#### consultant name extraction before validation

In [25]:
for key, value in texts_before_val.items():
    all_consultant_name = []
    consultant_str = ''
    for name in consultant_name_lst:
        if name.lower() in value:
            all_consultant_name.append(name)
            name_combine = name
            if len(name_combine.split()) > 1:
                name_combine = name_combine.replace(" ", "_")
            consultant_str = consultant_str + name_combine + ' '
    consultants_all.append(consultant_str)
    metadata.loc[len(metadata)] = [key, all_consultant_name]

In [26]:
metadata.head()

,file_name,All_Consultant_Names
0,2013-12-10- Watino and Calais Second PCMP Repo...,"[TERA Environmental Consultants, Midwest Surve..."
1,2014 Post Construction Environmental Monitorin...,"[TERA Environmental Consultants, CH2M HILL Ene..."
2,2014-01-28 Keystone PCEM Year 3 - A3T3I4,[]
3,2014-01-28 Keystone PCEM Year 4 - A3T3H0,"[Alberta Innovates Technology Futures, Alta Ra..."
4,2014-01-31- Cheecham Kettle River First PCMP R...,[Golder Associates]


In [27]:
len(metadata)

249

#### consultant name extraction after validation

In [28]:
consultant_name_after_val = []
for key, value in texts.items():
    consultant_in_pdf = []
    for name in consultant_name_lst:
        if name.lower() in value:
            consultant_in_pdf.append(name)
    consultant_name_after_val.append(consultant_in_pdf)

In [29]:
metadata['Consultant_Names_After_Validation'] = consultant_name_after_val
metadata['Consultant_Name_on_First_Two_Pages'] = consultant_name_two_pages
metadata['Reference_Count'] = count_reference
metadata['All_Consultants_Count'] = metadata['All_Consultant_Names'].apply(len)
metadata['Consultants_After_Validation_Count'] = metadata['Consultant_Names_After_Validation'].apply(len)
metadata['Consultant_Name_on_First_Two_Pages_Count'] = metadata['Consultant_Name_on_First_Two_Pages'].apply(len)
metadata['Eco_Words'] = eco_list
metadata['Bio_Words'] = bio_list
metadata['Author'] = author[0:249]
metadata['Creation_Date'] = creation_date[0:249]

In [30]:
metadata.head()

,file_name,All_Consultant_Names,Consultant_Names_After_Validation,Consultant_Name_on_First_Two_Pages,Reference_Count,All_Consultants_Count,Consultants_After_Validation_Count,Consultant_Name_on_First_Two_Pages_Count,Eco_Words,Bio_Words,Author,Creation_Date
0,2013-12-10- Watino and Calais Second PCMP Repo...,"[TERA Environmental Consultants, Midwest Surve...",[TERA Environmental Consultants],[],3,2,1,0,[economic],[],woodp,D:20131210110901-07'00'
1,2014 Post Construction Environmental Monitorin...,"[TERA Environmental Consultants, CH2M HILL Ene...","[TERA Environmental Consultants, CH2M HILL Ene...",[CH2M HILL Energy],4,2,2,1,"[economic, ecological, ecosystem]",[biogeoclimatic],woodp,D:20131210110901-07'00'
2,2014-01-28 Keystone PCEM Year 3 - A3T3I4,[],[],[],0,0,0,0,[],[],Aleksandra.Holod@ch2m.com;christopher.duits@ch...,D:20150120114914-07'00'
3,2014-01-28 Keystone PCEM Year 4 - A3T3H0,"[Alberta Innovates Technology Futures, Alta Ra...","[Alberta Innovates Technology Futures, Alta Ra...",[],6,4,3,0,"[ecology, ecosystems, ecoregion, ecological]",[biodiversity],Aleksandra.Holod@ch2m.com;christopher.duits@ch...,D:20150120114914-07'00'
4,2014-01-31- Cheecham Kettle River First PCMP R...,[Golder Associates],[Golder Associates],[],2,1,1,0,[ecosystem],"[biophysical, bio-engineering, biodegradable]",woodp,D:20140128151739-07'00'


In [31]:
vectorizer = TfidfVectorizer(lowercase = True, token_pattern = r'\S+') #reference: https://stackoverflow.com/questions/50502999/token-pattern-for-numbers-in-tfidfvectorizer-sklearn-in-python
#vectorizer = TfidfVectorizer() #Tfidf vectorizer to calculate tfidf scores for the consultants
X = vectorizer.fit_transform(consultants_all) #Calculate tfidf for the consultants set
files_map = {}
for idx, value in enumerate(text_files):
    files_map[idx] = value

In [32]:
#Create dataframe to store tfidf scores against text files
feature_names = vectorizer.get_feature_names()
corpus_index = [n for n in files_map]
df = pd.DataFrame(X.T.todense(), index=feature_names, columns=corpus_index)
df.columns = text_files

In [33]:
#Get top 3 consultants by tfidf scores
map_top_consultants_byfreq = {}
map_top_consultants_bytfidf = {}
for file in text_files:
    loc = df[file].to_dict()
    loc = {k:v for k,v in loc.items() if v != 0}
    top_3_tfidf = sorted(loc, key=loc.get, reverse=True)
    map_top_consultants_bytfidf[file] = top_3_tfidf

In [34]:
import numpy as np
metadata["Top_Consultants_tfidf"] = ""

In [35]:
for index, row in metadata.iterrows():
    metadata.at[index,'Top_Consultants_tfidf'] = str(map_top_consultants_bytfidf[row['file_name']]) #Add to dataframe

In [36]:
metadata.head(20)

,file_name,All_Consultant_Names,Consultant_Names_After_Validation,Consultant_Name_on_First_Two_Pages,Reference_Count,All_Consultants_Count,Consultants_After_Validation_Count,Consultant_Name_on_First_Two_Pages_Count,Eco_Words,Bio_Words,Author,Creation_Date,Top_Consultants_tfidf
0,2013-12-10- Watino and Calais Second PCMP Repo...,"[TERA Environmental Consultants, Midwest Surve...",[TERA Environmental Consultants],[],3,2,1,0,[economic],[],woodp,D:20131210110901-07'00',"['midwest_surveys_inc', 'tera_environmental_co..."
1,2014 Post Construction Environmental Monitorin...,"[TERA Environmental Consultants, CH2M HILL Ene...","[TERA Environmental Consultants, CH2M HILL Ene...",[CH2M HILL Energy],4,2,2,1,"[economic, ecological, ecosystem]",[biogeoclimatic],woodp,D:20131210110901-07'00',"['ch2m_hill_energy', 'tera_environmental_consu..."
2,2014-01-28 Keystone PCEM Year 3 - A3T3I4,[],[],[],0,0,0,0,[],[],Aleksandra.Holod@ch2m.com;christopher.duits@ch...,D:20150120114914-07'00',[]
3,2014-01-28 Keystone PCEM Year 4 - A3T3H0,"[Alberta Innovates Technology Futures, Alta Ra...","[Alberta Innovates Technology Futures, Alta Ra...",[],6,4,3,0,"[ecology, ecosystems, ecoregion, ecological]",[biodiversity],Aleksandra.Holod@ch2m.com;christopher.duits@ch...,D:20150120114914-07'00',"['alta_rangeland_services_ltd', 'gramineae_ser..."
4,2014-01-31- Cheecham Kettle River First PCMP R...,[Golder Associates],[Golder Associates],[],2,1,1,0,[ecosystem],"[biophysical, bio-engineering, biodegradable]",woodp,D:20140128151739-07'00',['golder_associates']
5,6873_Year 1 PCEM Pembina Beatton River - A3T4K3,"[TERA Environmental Consultants, Mentiga Pedol...",[TERA Environmental Consultants],[TERA Environmental Consultants],3,2,1,1,[ecosystem],[biogeoclimatic],woodp,D:20140128151739-07'00',"['mentiga_pedology_consultants_ltd', 'tera_env..."
6,A0L2F6 - 002 Ekwan Post Construction Monitorin...,[AXYS Environmental Consulting Ltd],[AXYS Environmental Consulting Ltd],[AXYS Environmental Consulting Ltd],0,1,1,1,"[ecologist, ecology]",[],woodp,D:20140128143003-07'00',['axys_environmental_consulting_ltd']
7,A0L3K4 - Kwoen Facilities and Re-injection Ext...,"[Jacques Whitford, AMEC Earth & Environmental]","[Jacques Whitford, AMEC Earth & Environmental]",[Jacques Whitford],2,2,2,1,[ecosystems],[biogeoclimatic],woodp,D:20140128143003-07'00',"['amec_earth_&_environmental', 'jacques_whitfo..."
8,A0T5H7 - Kwoen Re-injection Pipeline - 3rd Yea...,[],[],[],0,0,0,0,[],[biogeoclimatic],woodp,D:20140131093016-07'00',[]
9,A0U3V0 - 002 Post Construction Monitoring - Au...,[AXYS Environmental Consulting Ltd],[AXYS Environmental Consulting Ltd],[AXYS Environmental Consulting Ltd],2,1,1,1,"[ecologist, ecology, ecosystems, ecosystem]",[biogeoclimatic],woodp,D:20140131093016-07'00',['axys_environmental_consulting_ltd']


In [37]:
df1 = pd.read_csv('G:/Post Construction/metadata csvs/metadata_pcmr_Feb19.csv', encoding= 'unicode_escape', usecols = ['file_name','Company','Project_Name'])

In [38]:
new_df = metadata.merge(df1, on='file_name', how='left')

In [39]:
new_df.head()

,file_name,All_Consultant_Names,Consultant_Names_After_Validation,Consultant_Name_on_First_Two_Pages,Reference_Count,All_Consultants_Count,Consultants_After_Validation_Count,Consultant_Name_on_First_Two_Pages_Count,Eco_Words,Bio_Words,Author,Creation_Date,Top_Consultants_tfidf,Company,Project_Name
0,2013-12-10- Watino and Calais Second PCMP Repo...,"[TERA Environmental Consultants, Midwest Surve...",[TERA Environmental Consultants],[],3,2,1,0,[economic],[],woodp,D:20131210110901-07'00',"['midwest_surveys_inc', 'tera_environmental_co...",NOVA Gas Transmission Ltd.,2010-09-20 - Application for the Watino Crosso...
1,2014 Post Construction Environmental Monitorin...,"[TERA Environmental Consultants, CH2M HILL Ene...","[TERA Environmental Consultants, CH2M HILL Ene...",[CH2M HILL Energy],4,2,2,1,"[economic, ecological, ecosystem]",[biogeoclimatic],woodp,D:20131210110901-07'00',"['ch2m_hill_energy', 'tera_environmental_consu...",ARC Resources Ltd.,2008-07-09 - Application for Saddle Hills Pipe...
2,2014-01-28 Keystone PCEM Year 3 - A3T3I4,[],[],[],0,0,0,0,[],[],Aleksandra.Holod@ch2m.com;christopher.duits@ch...,D:20150120114914-07'00',[],TransCanada Keystone Pipeline GP Ltd.,2007-11-23 - Application for the Cushing Expan...
3,2014-01-28 Keystone PCEM Year 4 - A3T3H0,"[Alberta Innovates Technology Futures, Alta Ra...","[Alberta Innovates Technology Futures, Alta Ra...",[],6,4,3,0,"[ecology, ecosystems, ecoregion, ecological]",[biodiversity],Aleksandra.Holod@ch2m.com;christopher.duits@ch...,D:20150120114914-07'00',"['alta_rangeland_services_ltd', 'gramineae_ser...",TransCanada Keystone Pipeline GP Ltd.,2006-12-12 - Application for the Keystone Pipe...
4,2014-01-31- Cheecham Kettle River First PCMP R...,[Golder Associates],[Golder Associates],[],2,1,1,0,[ecosystem],"[biophysical, bio-engineering, biodegradable]",woodp,D:20140128151739-07'00',['golder_associates'],NOVA Gas Transmission Ltd.,2011-06-23 - Application for Cheecham - Kettle...


In [40]:
new_df.to_csv('consultant_name_Apr16.csv', encoding = 'utf-8-sig')

In [ ]:
eco_words_unique = metadata['Eco_Words'].to_list()
eco_words_flat_list = [item for sublist in eco_words_unique for item in sublist]
#print(eco_words_flat_list)
eco_words_unique_set = set(eco_words_flat_list)
bio_words_unique = metadata['Bio_Words'].to_list()
bio_words_flat_list = [item for sublist in bio_words_unique for item in sublist]
bio_words_unique_set = set(bio_words_flat_list)
print(eco_words_unique_set)
print(bio_words_unique_set)

#### word2vec model

In [ ]:
import re, string 
import sys
import os
import codecs
import pandas as pd   
from collections import defaultdict
import spacy
import gensim
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
from gensim.models import Word2Vec
%matplotlib inline
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
import en_core_web_sm
nlp = en_core_web_sm.load()
nlp.max_length = 6000000 #spaCy cannot process more than 1 million characters at once. Therefore nlp.max.length has to be changed as per the length of the text fed into the spaCy library functions

In [ ]:
STOPWORDS = set(stopwords.words('english'))

In [ ]:
nlp = en_core_web_sm.load(disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    # Remove a sentence if it is only one word long
    if len(text) > 2:
        return ' '.join(word for word in text.split() if word not in STOPWORDS)

In [ ]:
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [ ]:
list_documents = []
file_names = []
files = os.listdir("G:/Post Construction/PDF_Final_text") #directory path where ESAs as text files are saved
for file in files:
    if file.endswith('.txt'):
        with codecs.open("G:/Post Construction/PDF_Final_text/" + file,'r', encoding='utf-8-sig') as corpus: #directory path where ESAs as text files are saved
            file_names.append(file)
            input_str = corpus.read()
            input_str = clean_text(input_str)
#             input_str = lemmatizer(input_str)
#             input_str = input_str.replace('-PRON-', '')
            list_documents.append(input_str)

In [ ]:
len(file_names)

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))

word_tokens = list(sent_to_words(list_documents))

In [ ]:
# First of all, we will build the bigram and trigram model
bigram = gensim.models.Phrases(word_tokens, min_count = 20, threshold = 16) # higher threshold means fewer phrases.
trigram = gensim.models.Phrases(bigram[word_tokens], threshold = 16)

#min_count: ignore all words and bigrams with total collected count lower than this
#threshold represents a score threshold for forming the phrases (higher means fewer phrases). A phrase of words a followed by b is accepted if the score of the phrase is greater than threshold

# Faster way to get a sentence clubbed as a trigram/bigram 
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(bigram_mod[word_tokens[0]]) # this will print unigrams, bigrams as well as trigrams. Bigrams are two words frequently occuring together in the documents and trigram is three words frequently occuring together.

In [ ]:
def make_bigrams(tokens):
    return [bigram_mod[doc] for doc in tokens]

In [ ]:
def make_trigrams(tokens):
    return [trigram_mod[bigram_mod[doc]] for doc in tokens]

In [ ]:
sentences = make_bigrams(word_tokens)
#sentences = make_trigrams(word_tokens)
print(type(sentences))

In [ ]:
w2v_model = Word2Vec(min_count = 45,
                     window = 10,
                     size = 100,
                     workers = 2)

In [ ]:
w2v_model.build_vocab(sentences) # if this throws error, then try the below code with update parameter
#w2v_model.build_vocab(sentences, update = True)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)
w2v_model.init_sims(replace=True)

In [ ]:
eco_words_lst_word2vec = ['ecosites','ecoregion', 'ecozones', 'ecosystem', 'ecodistricts']
ecoregion_lst = ['agricultural', 'cropland', 'native prairie', 'grassland', 'residential', 'commercial', 'industrial', 'boreal forest', 'mixed forest', 'national park', 'provincial park', 'green space','wetlands','Watercourse','Tundra','Delta','mountains','mountainous']
eco_words_lst_word2vec.extend(ecoregion_lst)

In [ ]:
eco_words_lst_word2vec

In [ ]:
root_word_dict = {}
for root_word in eco_words_lst_word2vec:
    try:
        context_words = w2v_model.wv.most_similar(positive = [root_word],topn = 20)
        root_word_dict[root_word] = context_words
    except:
        root_word_dict[root_word] = 'The word is not in vocabulary'

In [ ]:
# Convert the dictionary to dataframe
word2vec_df = pd.DataFrame.from_dict(root_word_dict)

In [ ]:
word2vec_df.to_csv(r'G:\Post Construction\Ad-hoc Requests\Consultant_Name_Extraction\word2vec_bigram_v1.csv', encoding = 'utf-8-sig')

In [ ]:
def tsne_plot(model):
    "Create TSNE model and plot it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(18, 18)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
tsne_plot(w2v_model)